In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

# Data

In [5]:
from google.colab import drive

drive.mount("/content/gdrive")

MessageError: ignored

### Data loading 

In [ ]:
covid_19 = pd.read_csv("gdrive/My Drive/Project SOAR/COVID-19-global-data.csv")
covid_19.tail() #covid record start from 22 Janv 2020

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
181995,2021-10-28,Zimbabwe,NaN,132880,0.0,4675
181996,2021-10-29,Zimbabwe,NaN,132926,0.0,4675
181997,2021-10-30,Zimbabwe,NaN,132926,0.0,4675
181998,2021-10-31,Zimbabwe,NaN,132977,0.0,4678
181999,2021-11-01,Zimbabwe,NaN,133007,0.0,4679


In [ ]:
covid_19.shape

(182000, 6)

Our first training period

In [ ]:
noLockdown = covid_19['Date'] < "2020-03-16"
covid_19_noLockdown = covid_19[noLockdown]
covid_19_noLockdown.tail()

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
181399,2020-03-11,Zimbabwe,NaN,0,0.0,0
181400,2020-03-12,Zimbabwe,NaN,0,0.0,0
181401,2020-03-13,Zimbabwe,NaN,0,0.0,0
181402,2020-03-14,Zimbabwe,NaN,0,0.0,0
181403,2020-03-15,Zimbabwe,NaN,0,0.0,0


In [ ]:
covid_19_noLockdown.dtypes #date format a changer et uniformiser les deux dataframes  / province à drop / 

Date               object
Country/Region     object
Province/State     object
Confirmed           int64
Recovered         float64
Deaths              int64
dtype: object

In [ ]:
covid_19_noLockdown['Date'] = pd.to_datetime(covid_19_noLockdown['Date'], infer_datetime_format=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
covid_19['Date'] = pd.to_datetime(covid_19['Date'], infer_datetime_format=True)
covid_19['Date'].head() #Date formatted 

0   2020-01-22
1   2020-01-23
2   2020-01-24
3   2020-01-25
4   2020-01-26
Name: Date, dtype: datetime64[ns]

In [ ]:
covid_19_noLockdown.dtypes

Date              datetime64[ns]
Country/Region            object
Province/State            object
Confirmed                  int64
Recovered                float64
Deaths                     int64
dtype: object

In [ ]:
covid_19.dtypes

Date              datetime64[ns]
Country/Region            object
Province/State            object
Confirmed                  int64
Recovered                float64
Deaths                     int64
dtype: object

In [ ]:
# dropping province 
covid_19_noLockdown.drop(['Province/State'], axis=1, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
covid_19_noLockdown.head()

,Date,Country/Region,Confirmed,Recovered,Deaths
0,2020-01-22,Afghanistan,0,0.0,0
1,2020-01-23,Afghanistan,0,0.0,0
2,2020-01-24,Afghanistan,0,0.0,0
3,2020-01-25,Afghanistan,0,0.0,0
4,2020-01-26,Afghanistan,0,0.0,0


a. For the Covid-19 dataset: date-time VS number of cases per country

Let's normalize the data before model fitting to boost the performance.

In [ ]:
# How to normalise date???

We need to normalise the number of cases. 

## Let's build the model

We will build the LSTM with a number of neurons and 3 to 4 hidden layers. We will use the MSE loss function and the Adam stochastic gradient descent optimizer (define it).

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(covid_19_noLockdown['Date'], covid_19_noLockdown['Confirmed'], epochs=100, batch_size=1, verbose=2)

### let s predict 

We will use the date from 16 March to 30 May

In [ ]:
one = covid_19['Date'] >= "2020-03-16"
two = covid_19['Date'] <= "2020-06-15"
after_Lockdown = covid_19[(one) & (two)]
after_Lockdown.tail()

,Date,Country/Region,Province/State,Confirmed,Recovered,Deaths
181491,2020-06-11,Zimbabwe,NaN,332,51.0,4
181492,2020-06-12,Zimbabwe,NaN,343,51.0,4
181493,2020-06-13,Zimbabwe,NaN,356,54.0,4
181494,2020-06-14,Zimbabwe,NaN,383,54.0,4
181495,2020-06-15,Zimbabwe,NaN,387,54.0,4


In [ ]:
predicted_cases = model.predict(after_Lockdown)
predicted_cases = sc.inverse_transform(predicted_cases) #why this step is required?